In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import spearmanr, pearsonr, shapiro
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

# 環境変数読み込み
load_dotenv()
database_url = os.getenv('DATABASE_URL')
engine = create_engine(database_url)
print("✅ データベース接続完了")

✅ データベース接続完了


In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import spearmanr, pearsonr, shapiro
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

# 環境変数読み込み
load_dotenv()
database_url = os.getenv('DATABASE_URL')
engine = create_engine(database_url)
print('✅ データベース接続完了')

## 1. データ読み込み

In [ ]:
# セッションデータ
sessions_query = """
    SELECT id, current_phase
    FROM sessions
    WHERE current_phase = 'completed'
"""
sessions_df = pd.read_sql(sessions_query, engine)

# 回答データ（phase = 'final_check' のみ）
responses_query = """
    SELECT 
        r.id, r.session_id, r.problem_id,
        r.confidence, r.challenge,
        r.ai_predicted_confidence, r.ai_predicted_challenge,
        r.phase,
        p.knowledge_component, p.description_main, p.description_sub
    FROM responses r
    JOIN problems p ON r.problem_id = p.id
    WHERE r.session_id IN (SELECT id FROM sessions WHERE current_phase = 'completed')
      AND r.phase = 'final_check'
      AND r.confidence IS NOT NULL
      AND r.challenge IS NOT NULL
      AND r.ai_predicted_confidence IS NOT NULL
      AND r.ai_predicted_challenge IS NOT NULL
"""
df = pd.read_sql(responses_query, engine)

# GAP計算
df['human_gap'] = df['confidence'] - df['challenge']
df['ai_gap'] = df['ai_predicted_confidence'] - df['ai_predicted_challenge']
df['gap_difference'] = (df['human_gap'] - df['ai_gap']).abs()

print(f'読み込み完了: {len(df)}件の回答データ (phase=final_check のみ)')

読み込み完了: 300件の回答データ (phase=final_check のみ)


## 2. 基本統計（考察.md 1.2節の検証）

期待値:
- 問題数: 35問
- 被験者: 10人
- セッション数: 11
- 総回答数: 300件

In [16]:
print('=' * 50)
print('📊 基本統計')
print('=' * 50)
print(f'完了セッション数: {len(sessions_df)}')
print(f'総回答数: {len(df)}')
print(f'問題数: {df["problem_id"].nunique()}')
print(f'ユーザー数: {df["session_id"].nunique()}')
print('=' * 50)

📊 基本統計
完了セッション数: 11
総回答数: 300
問題数: 35
ユーザー数: 10


## 3. 評価関数の定義

In [5]:
def calculate_metrics(actual, predicted):
    """評価指標を計算"""
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    # 相関係数
    r, p_value = pearsonr(actual, predicted)
    r2 = r ** 2
    
    # MAE
    mae = np.abs(actual - predicted).mean()
    
    # 一致率
    exact_match = (actual == predicted).sum() / len(actual) * 100
    within_1 = (np.abs(actual - predicted) <= 1).sum() / len(actual) * 100
    within_2 = (np.abs(actual - predicted) <= 2).sum() / len(actual) * 100
    
    return {
        'r': r,
        'p_value': p_value,
        'r2': r2,
        'mae': mae,
        'exact_match': exact_match,
        'within_1': within_1,
        'within_2': within_2
    }

def print_metrics(name, metrics):
    """評価指標を表示"""
    print(f'\n{name}:')
    print(f'  相関係数 (r): {metrics["r"]:.3f}')
    print(f'  決定係数 (R²): {metrics["r2"]:.3f}')
    print(f'  MAE: {metrics["mae"]:.2f}')
    print(f'  完全一致率: {metrics["exact_match"]:.1f}%')
    print(f'  ±1以内: {metrics["within_1"]:.1f}%')

## 4. ベースライン計算（Leave-One-Out方式）

In [6]:
# Leave-One-Out方式でベースラインを計算

def calc_loo_baselines(df, value_col):
    """Leave-One-Out方式でベースラインを計算"""
    result_df = df.copy()
    
    # 全体平均（自分を除く）
    global_mean = df[value_col].mean()
    result_df['global_mean'] = global_mean
    
    # 問題平均（LOO）
    def calc_problem_loo(row):
        same_problem = df[(df['problem_id'] == row['problem_id']) & (df['session_id'] != row['session_id'])]
        if len(same_problem) > 0:
            return same_problem[value_col].mean()
        else:
            return df[df['session_id'] != row['session_id']][value_col].mean()
    
    result_df['problem_mean'] = df.apply(calc_problem_loo, axis=1)
    
    # ユーザー平均（LOO）
    def calc_user_loo(row):
        other_problems = df[(df['session_id'] == row['session_id']) & (df['problem_id'] != row['problem_id'])]
        if len(other_problems) > 0:
            return other_problems[value_col].mean()
        else:
            return df[df['session_id'] != row['session_id']][value_col].mean()
    
    result_df['user_mean'] = df.apply(calc_user_loo, axis=1)
    
    return result_df

print('ベースライン計算関数を定義しました')

ベースライン計算関数を定義しました


## 5. 自信度予測の検証（考察.md 2.2節）

期待値:
- LLM: MAE=1.09, r=0.600, R²=0.360, 完全一致=32.7%, ±1以内=63.7%
- 全体平均: MAE=1.65, r=0, 完全一致=8%, ±1以内=22%
- 問題平均: MAE=1.35, r=0.467, 完全一致=16%, ±1以内=48%
- ユーザー平均: MAE=1.30, r=0.457, 完全一致=21.3%, ±1以内=44.7%

In [17]:
print('=' * 50)
print('📊 自信度予測の検証')
print('=' * 50)

# 自信度のベースライン計算
conf_df = calc_loo_baselines(df, 'confidence')

# LLM予測
llm_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['ai_predicted_confidence'])
print_metrics('LLM予測', llm_conf_metrics)

# 全体平均
global_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['global_mean'])
print_metrics('全体平均', global_conf_metrics)

# 問題平均
problem_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['problem_mean'])
print_metrics('問題平均', problem_conf_metrics)

# ユーザー平均
user_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['user_mean'])
print_metrics('ユーザー平均', user_conf_metrics)

📊 自信度予測の検証

LLM予測:
  相関係数 (r): 0.600
  決定係数 (R²): 0.360
  MAE: 1.09
  完全一致率: 32.3%
  ±1以内: 70.7%

全体平均:
  相関係数 (r): nan
  決定係数 (R²): nan
  MAE: 1.65
  完全一致率: 0.0%
  ±1以内: 23.3%

問題平均:
  相関係数 (r): 0.440
  決定係数 (R²): 0.194
  MAE: 1.35
  完全一致率: 3.3%
  ±1以内: 47.0%

ユーザー平均:
  相関係数 (r): 0.449
  決定係数 (R²): 0.201
  MAE: 1.30
  完全一致率: 10.0%
  ±1以内: 46.0%


## 6. 挑戦度予測の検証（考察.md 2.3節）

期待値:
- LLM: MAE=1.23, r=0.371
- 全体平均: MAE=1.18
- 問題平均: MAE=0.94, r=0.500
- ユーザー平均: MAE=1.01, r=0.347

In [18]:
print('=' * 50)
print('📊 挑戦度予測の検証')
print('=' * 50)

# 挑戦度のベースライン計算
chal_df = calc_loo_baselines(df, 'challenge')

# LLM予測
llm_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['ai_predicted_challenge'])
print_metrics('LLM予測', llm_chal_metrics)

# 全体平均
global_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['global_mean'])
print_metrics('全体平均', global_chal_metrics)

# 問題平均
problem_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['problem_mean'])
print_metrics('問題平均', problem_chal_metrics)

# ユーザー平均
user_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['user_mean'])
print_metrics('ユーザー平均', user_chal_metrics)

📊 挑戦度予測の検証

LLM予測:
  相関係数 (r): 0.371
  決定係数 (R²): 0.138
  MAE: 1.23
  完全一致率: 23.7%
  ±1以内: 66.3%

全体平均:
  相関係数 (r): nan
  決定係数 (R²): nan
  MAE: 1.18
  完全一致率: 0.0%
  ±1以内: 47.3%

問題平均:
  相関係数 (r): 0.500
  決定係数 (R²): 0.250
  MAE: 0.94
  完全一致率: 3.7%
  ±1以内: 67.0%

ユーザー平均:
  相関係数 (r): 0.347
  決定係数 (R²): 0.120
  MAE: 1.01
  完全一致率: 11.7%
  ±1以内: 56.3%


## 7. GAP予測の検証（考察.md 2.4節）

期待値:
- LLM: MAE=2.11, r=0.532, R²=0.283, 完全一致=21.7%, ±1以内=42.3%
- 全体平均: MAE=2.68, 完全一致=4%, ±1以内=7%
- 問題平均: MAE=2.07, r=0.513
- ユーザー平均: MAE=2.20, r=0.403

In [19]:
print('=' * 50)
print('📊 GAP予測の検証')
print('=' * 50)

# GAPのベースライン計算
gap_df = calc_loo_baselines(df, 'human_gap')

# LLM予測
llm_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['ai_gap'])
print_metrics('LLM予測', llm_gap_metrics)

# 全体平均
global_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['global_mean'])
print_metrics('全体平均', global_gap_metrics)

# 問題平均
problem_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['problem_mean'])
print_metrics('問題平均', problem_gap_metrics)

# ユーザー平均
user_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['user_mean'])
print_metrics('ユーザー平均', user_gap_metrics)

📊 GAP予測の検証

LLM予測:
  相関係数 (r): 0.532
  決定係数 (R²): 0.283
  MAE: 2.11
  完全一致率: 21.7%
  ±1以内: 42.3%

全体平均:
  相関係数 (r): nan
  決定係数 (R²): nan
  MAE: 2.68
  完全一致率: 0.0%
  ±1以内: 7.0%

問題平均:
  相関係数 (r): 0.513
  決定係数 (R²): 0.263
  MAE: 2.07
  完全一致率: 1.7%
  ±1以内: 35.0%

ユーザー平均:
  相関係数 (r): 0.403
  決定係数 (R²): 0.162
  MAE: 2.20
  完全一致率: 10.0%
  ±1以内: 28.3%


## 8. 混合効果モデル分析（考察.md 3.2節）

期待値:
- 問題効果 (ICC): 27.1%
- ユーザー効果 (ICC): 10.2%
- 残差: 62.7%
- LLM係数 (β): 0.524, p < 0.001
- 周辺R²: 10.0%
- 条件付きR²: 37.3%

In [20]:
print('=' * 50)
print('📊 混合効果モデル分析')
print('=' * 50)

# データ準備
model_df = df[['human_gap', 'ai_gap', 'problem_id', 'session_id']].copy()
model_df = model_df.rename(columns={'session_id': 'user_id'})
model_df = model_df.dropna()

# Full Model
full_model = smf.mixedlm(
    'human_gap ~ ai_gap', 
    data=model_df, 
    groups=model_df['user_id'],
    re_formula='~1',
    vc_formula={'problem_id': '0 + C(problem_id)'}
)
full_result = full_model.fit()

# 分散成分の抽出
sigma2_user = full_result.cov_re.iloc[0, 0]
sigma2_problem = full_result.vcomp[0] if len(full_result.vcomp) > 0 else 0
sigma2_residual = full_result.scale
total_variance = sigma2_problem + sigma2_user + sigma2_residual

# ICC
icc_problem = sigma2_problem / total_variance
icc_user = sigma2_user / total_variance
icc_residual = sigma2_residual / total_variance

print(f'\n分散成分:')
print(f'  問題効果 (ICC): {icc_problem:.1%}')
print(f'  ユーザー効果 (ICC): {icc_user:.1%}')
print(f'  残差: {icc_residual:.1%}')

# 固定効果
beta_ai = full_result.params['ai_gap']
p_ai = full_result.pvalues['ai_gap']

print(f'\n固定効果:')
print(f'  LLM係数 (β): {beta_ai:.3f}')
print(f'  p値: {p_ai:.4g}')

# R² (Nakagawa & Schielzeth, 2013)
fixed_pred = full_result.params['Intercept'] + full_result.params['ai_gap'] * model_df['ai_gap']
var_fixed = fixed_pred.var()
var_random = sigma2_problem + sigma2_user
denom = var_fixed + var_random + sigma2_residual

marginal_r2 = var_fixed / denom
conditional_r2 = (var_fixed + var_random) / denom

print(f'\nR²:')
print(f'  周辺R² (固定効果のみ): {marginal_r2:.1%}')
print(f'  条件付きR² (全効果): {conditional_r2:.1%}')

📊 混合効果モデル分析

分散成分:
  問題効果 (ICC): 47.0%
  ユーザー効果 (ICC): 17.4%
  残差: 35.6%

固定効果:
  LLM係数 (β): 0.617
  p値: 1.2e-24

R²:
  周辺R² (固定効果のみ): 26.7%
  条件付きR² (全効果): 73.9%


## 9. 残差診断（考察.md 3.3節）

期待値:
- 平均: -0.0000
- 歪度: -0.33
- 尖度: -0.21
- W統計量: 0.9875, p = 0.011

In [21]:
print('=' * 50)
print('📊 残差診断')
print('=' * 50)

# 残差の計算
residuals = model_df['human_gap'] - full_result.fittedvalues

print(f'\n残差の統計量:')
print(f'  平均: {residuals.mean():.4f}')
print(f'  標準偏差: {residuals.std():.4f}')
print(f'  歪度: {stats.skew(residuals):.4f}')
print(f'  尖度: {stats.kurtosis(residuals):.4f}')

# Shapiro-Wilk検定
shapiro_stat, shapiro_p = shapiro(residuals)
print(f'\nShapiro-Wilk検定:')
print(f'  W = {shapiro_stat:.4f}')
print(f'  p = {shapiro_p:.4f}')

📊 残差診断

残差の統計量:
  平均: -0.0000
  標準偏差: 1.0011
  歪度: -0.3282
  尖度: -0.2109

Shapiro-Wilk検定:
  W = 0.9875
  p = 0.0109


## 10. ピアソン vs スピアマン相関の比較（考察.md 3.4節）

期待値:
- 自信度: Pearson=0.600, Spearman=0.576, 差=0.024
- 挑戦度: Pearson=0.371, Spearman=0.360, 差=0.011
- GAP: Pearson=0.532, Spearman=0.507, 差=0.025

In [22]:
print('=' * 50)
print('📊 ピアソン vs スピアマン相関の比較')
print('=' * 50)

comparisons = [
    ('自信度', 'confidence', 'ai_predicted_confidence'),
    ('挑戦度', 'challenge', 'ai_predicted_challenge'),
    ('GAP', 'human_gap', 'ai_gap'),
]

print(f'\n{"指標":<10} {"ピアソン(r)":<12} {"スピアマン(ρ)":<14} {"差":<8} {"判定"}')
print('-' * 55)

for name, human_col, ai_col in comparisons:
    pearson_r, _ = pearsonr(df[human_col], df[ai_col])
    spearman_r, _ = spearmanr(df[human_col], df[ai_col])
    diff = abs(pearson_r - spearman_r)
    judge = '✅ 一致' if diff < 0.05 else '⚠️ 乖離'
    print(f'{name:<10} {pearson_r:<12.3f} {spearman_r:<14.3f} {diff:<8.3f} {judge}')

📊 ピアソン vs スピアマン相関の比較

指標         ピアソン(r)      スピアマン(ρ)       差        判定
-------------------------------------------------------
自信度        0.600        0.585          0.015    ✅ 一致
挑戦度        0.371        0.377          0.006    ✅ 一致
GAP        0.532        0.530          0.001    ✅ 一致


## 11. 人間の自信度と挑戦度の相関

In [23]:
print('=' * 50)
print('📊 人間の自信度と挑戦度の相関')
print('=' * 50)

conf_chal_r, conf_chal_p = pearsonr(df['confidence'], df['challenge'])
print(f'\nPearson相関係数: r = {conf_chal_r:.4f}')
print(f'p値: {conf_chal_p:.4g}')
print(f'決定係数 (R²): {conf_chal_r**2:.4f}')

if abs(conf_chal_r) >= 0.7:
    strength = '強い'
elif abs(conf_chal_r) >= 0.4:
    strength = '中程度の'
else:
    strength = '弱い'
direction = '正の' if conf_chal_r > 0 else '負の'

print(f'\n解釈: {strength}{direction}相関')

📊 人間の自信度と挑戦度の相関

Pearson相関係数: r = -0.7429
p値: 6.923e-54
決定係数 (R²): 0.5519

解釈: 強い負の相関


## 12. 結果サマリー

In [24]:
print('='*60)
print('📋 考察.mdとの比較サマリー')
print('='*60)

print('\n【基本統計】')
print(f'  セッション数: {len(sessions_df)} (期待値: 11)')
print(f'  回答数: {len(df)} (期待値: 300)')
print(f'  問題数: {df["problem_id"].nunique()} (期待値: 35)')
print(f'  ユーザー数: {df["session_id"].nunique()} (期待値: 10)')

print('\n【自信度予測 - LLM】')
print(f'  MAE: {llm_conf_metrics["mae"]:.2f} (期待値: 1.09)')
print(f'  r: {llm_conf_metrics["r"]:.3f} (期待値: 0.600)')
print(f'  完全一致率: {llm_conf_metrics["exact_match"]:.1f}% (期待値: 32.7%)')

print('\n【挑戦度予測 - LLM】')
print(f'  MAE: {llm_chal_metrics["mae"]:.2f} (期待値: 1.23)')
print(f'  r: {llm_chal_metrics["r"]:.3f} (期待値: 0.371)')

print('\n【GAP予測 - LLM】')
print(f'  MAE: {llm_gap_metrics["mae"]:.2f} (期待値: 2.11)')
print(f'  r: {llm_gap_metrics["r"]:.3f} (期待値: 0.532)')

print('\n【混合効果モデル】')
print(f'  問題効果 (ICC): {icc_problem:.1%} (期待値: 27.1%)')
print(f'  ユーザー効果 (ICC): {icc_user:.1%} (期待値: 10.2%)')
print(f'  LLM係数 (β): {beta_ai:.3f} (期待値: 0.524)')
print(f'  周辺R²: {marginal_r2:.1%} (期待値: 10.0%)')

📋 考察.mdとの比較サマリー

【基本統計】
  セッション数: 11 (期待値: 11)
  回答数: 300 (期待値: 300)
  問題数: 35 (期待値: 35)
  ユーザー数: 10 (期待値: 10)

【自信度予測 - LLM】
  MAE: 1.09 (期待値: 1.09)
  r: 0.600 (期待値: 0.600)
  完全一致率: 32.3% (期待値: 32.7%)

【挑戦度予測 - LLM】
  MAE: 1.23 (期待値: 1.23)
  r: 0.371 (期待値: 0.371)

【GAP予測 - LLM】
  MAE: 2.11 (期待値: 2.11)
  r: 0.532 (期待値: 0.532)

【混合効果モデル】
  問題効果 (ICC): 47.0% (期待値: 27.1%)
  ユーザー効果 (ICC): 17.4% (期待値: 10.2%)
  LLM係数 (β): 0.617 (期待値: 0.524)
  周辺R²: 26.7% (期待値: 10.0%)


## 13. 推薦システムの検証: AI GAP = 0 の効果

**仮説**: AIがGAP = 0（自信度 = 挑戦度）と予測した問題は、人間にとってもGAPが小さくなる（適切な問題である）

In [ ]:
print('=' * 60)
print('📊 推薦システムの検証: AI GAP = 0 の効果')
print('=' * 60)

# AI GAPが0のグループとそれ以外のグループを分ける
df_ai_gap_zero = df[df['ai_gap'] == 0]
df_ai_gap_nonzero = df[df['ai_gap'] != 0]

print(f'\n【データの分布】')
print(f'  AI GAP = 0 の回答数: {len(df_ai_gap_zero)} ({len(df_ai_gap_zero)/len(df)*100:.1f}%)')
print(f'  AI GAP ≠ 0 の回答数: {len(df_ai_gap_nonzero)} ({len(df_ai_gap_nonzero)/len(df)*100:.1f}%)')

# Human GAPの統計量を比較
print(f'\n【Human GAP の統計量】')
print(f'\n  AI GAP = 0 のグループ:')
print(f'    平均: {df_ai_gap_zero["human_gap"].mean():.3f}')
print(f'    標準偏差: {df_ai_gap_zero["human_gap"].std():.3f}')
print(f'    中央値: {df_ai_gap_zero["human_gap"].median():.3f}')
print(f'    |GAP|の平均: {df_ai_gap_zero["human_gap"].abs().mean():.3f}')

print(f'\n  AI GAP ≠ 0 のグループ:')
print(f'    平均: {df_ai_gap_nonzero["human_gap"].mean():.3f}')
print(f'    標準偏差: {df_ai_gap_nonzero["human_gap"].std():.3f}')
print(f'    中央値: {df_ai_gap_nonzero["human_gap"].median():.3f}')
print(f'    |GAP|の平均: {df_ai_gap_nonzero["human_gap"].abs().mean():.3f}')

In [ ]:
# 統計的検定
print('\n【統計的検定】')

# 1. |Human GAP| の比較 (Mann-Whitney U検定 - ノンパラメトリック)
from scipy.stats import mannwhitneyu, ttest_ind

abs_gap_zero = df_ai_gap_zero['human_gap'].abs()
abs_gap_nonzero = df_ai_gap_nonzero['human_gap'].abs()

# Mann-Whitney U検定
u_stat, u_p = mannwhitneyu(abs_gap_zero, abs_gap_nonzero, alternative='less')
print(f'\n  Mann-Whitney U検定 (|Human GAP|が小さいか):')
print(f'    U統計量: {u_stat:.2f}')
print(f'    p値 (片側): {u_p:.4f}')
print(f'    判定: {"✅ 有意 (p < 0.05)" if u_p < 0.05 else "❌ 有意でない"}')

# 2. Welchのt検定
t_stat, t_p = ttest_ind(abs_gap_zero, abs_gap_nonzero, equal_var=False)
print(f'\n  Welchのt検定 (|Human GAP|の平均比較):')
print(f'    t統計量: {t_stat:.2f}')
print(f'    p値 (両側): {t_p:.4f}')
print(f'    判定: {"✅ 有意 (p < 0.05)" if t_p < 0.05 else "❌ 有意でない"}')

# 効果量 (Cohen's d)
mean_diff = abs_gap_zero.mean() - abs_gap_nonzero.mean()
pooled_std = np.sqrt((abs_gap_zero.std()**2 + abs_gap_nonzero.std()**2) / 2)
cohens_d = mean_diff / pooled_std
print(f'\n  効果量 (Cohen\'s d): {cohens_d:.3f}')
if abs(cohens_d) < 0.2:
    effect_size = '効果なし'
elif abs(cohens_d) < 0.5:
    effect_size = '小'
elif abs(cohens_d) < 0.8:
    effect_size = '中'
else:
    effect_size = '大'
print(f'    解釈: {effect_size}')

In [ ]:
# AI GAPの値別にHuman GAPの|絶対値|を分析
print('\n【AI GAP値別の Human |GAP| 分析】')
print('-' * 50)

# AI GAPの各値に対して集計
ai_gap_analysis = df.groupby('ai_gap').agg({
    'human_gap': ['count', 'mean', 'std', lambda x: x.abs().mean()]
}).round(3)
ai_gap_analysis.columns = ['件数', 'Human GAP平均', 'Human GAP標準偏差', '|Human GAP|平均']
ai_gap_analysis = ai_gap_analysis.sort_index()

print(ai_gap_analysis.to_string())

# Human GAP = 0 の割合を計算
print('\n【AI GAP値別の Human GAP = 0 の割合】')
print('-' * 50)
for ai_gap_val in sorted(df['ai_gap'].unique()):
    subset = df[df['ai_gap'] == ai_gap_val]
    human_gap_zero_rate = (subset['human_gap'] == 0).sum() / len(subset) * 100
    human_gap_small_rate = (subset['human_gap'].abs() <= 1).sum() / len(subset) * 100
    print(f'  AI GAP = {ai_gap_val:>2}: Human GAP=0 {human_gap_zero_rate:>5.1f}%, |Human GAP|≤1 {human_gap_small_rate:>5.1f}% (n={len(subset)})')

In [ ]:
# 結論
print('\n' + '=' * 60)
print('📋 結論: AI GAP = 0 の推薦効果')
print('=' * 60)

print(f'''
【検証結果】
  AI GAP = 0 の場合:
    - |Human GAP|の平均: {abs_gap_zero.mean():.3f}
    - Human GAP = 0 の割合: {(df_ai_gap_zero["human_gap"] == 0).mean()*100:.1f}%
    
  AI GAP ≠ 0 の場合:
    - |Human GAP|の平均: {abs_gap_nonzero.mean():.3f}
    - Human GAP = 0 の割合: {(df_ai_gap_nonzero["human_gap"] == 0).mean()*100:.1f}%

【統計的検定の結果】
  Mann-Whitney U検定 p値: {u_p:.4f}
  効果量 (Cohen's d): {cohens_d:.3f} ({effect_size})

【解釈】
''')

if u_p < 0.05 and cohens_d < 0:
    print('  ✅ 仮説は支持された')
    print('  AI GAP = 0 と予測された問題は、人間にとってもGAPが有意に小さい')
    print('  → 推薦システムとして機能している可能性がある')
elif u_p >= 0.05:
    print('  ❌ 仮説は支持されなかった')
    print('  AI GAP = 0 と AI GAP ≠ 0 で、Human GAPに有意差がない')
else:
    print('  ⚠️ 逆の結果')
    print('  AI GAP = 0 の方が、Human GAPが大きい傾向がある')

## 1. データ読み込み

In [ ]:
# セッションデータ
sessions_query = """
    SELECT id, current_phase, created_at, completed_at
    FROM sessions
    WHERE current_phase = 'completed'
"""
sessions_df = pd.read_sql(sessions_query, engine)

# 回答データ
responses_query = """
    SELECT 
        r.id, r.session_id, r.problem_id,
        r.confidence, r.challenge,
        r.ai_predicted_confidence, r.ai_predicted_challenge,
        p.knowledge_component, p.description_main, p.description_sub
    FROM responses r
    JOIN problems p ON r.problem_id = p.id
    WHERE r.session_id IN (SELECT id FROM sessions WHERE current_phase = 'completed')
      AND r.confidence IS NOT NULL
      AND r.challenge IS NOT NULL
      AND r.ai_predicted_confidence IS NOT NULL
      AND r.ai_predicted_challenge IS NOT NULL
"""
df = pd.read_sql(responses_query, engine)

# GAP計算
df['human_gap'] = df['confidence'] - df['challenge']
df['ai_gap'] = df['ai_predicted_confidence'] - df['ai_predicted_challenge']
df['gap_difference'] = (df['human_gap'] - df['ai_gap']).abs()

print(f"読み込み完了: {len(df)}件の回答データ")

## 2. 基本統計（考察.md 1.2節の検証）

期待値:
- 問題数: 35問
- 被験者: 10人
- セッション数: 11
- 総回答数: 300件

In [ ]:
print("=" * 50)
print("📊 基本統計")
print("=" * 50)
print(f"完了セッション数: {len(sessions_df)}")
print(f"総回答数: {len(df)}")
print(f"問題数: {df['problem_id'].nunique()}")
print(f"ユーザー数: {df['session_id'].nunique()}")
print("=" * 50)

## 3. 評価関数の定義

In [ ]:
def calculate_metrics(actual, predicted):
    """評価指標を計算"""
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    # 相関係数
    r, p_value = pearsonr(actual, predicted)
    r2 = r ** 2
    
    # MAE
    mae = np.abs(actual - predicted).mean()
    
    # 一致率
    exact_match = (actual == predicted).sum() / len(actual) * 100
    within_1 = (np.abs(actual - predicted) <= 1).sum() / len(actual) * 100
    within_2 = (np.abs(actual - predicted) <= 2).sum() / len(actual) * 100
    
    return {
        'r': r,
        'p_value': p_value,
        'r2': r2,
        'mae': mae,
        'exact_match': exact_match,
        'within_1': within_1,
        'within_2': within_2
    }

def print_metrics(name, metrics):
    """評価指標を表示"""
    print(f"\n{name}:")
    print(f"  相関係数 (r): {metrics['r']:.3f}")
    print(f"  決定係数 (R²): {metrics['r2']:.3f}")
    print(f"  MAE: {metrics['mae']:.2f}")
    print(f"  完全一致率: {metrics['exact_match']:.1f}%")
    print(f"  ±1以内: {metrics['within_1']:.1f}%")

## 4. ベースライン計算（Leave-One-Out方式）

In [ ]:
# Leave-One-Out方式でベースラインを計算

def calc_loo_baselines(df, value_col):
    """Leave-One-Out方式でベースラインを計算"""
    result_df = df.copy()
    
    # 全体平均（自分を除く）
    global_mean = df[value_col].mean()
    result_df['global_mean'] = global_mean
    
    # 問題平均（LOO）
    def calc_problem_loo(row):
        same_problem = df[(df['problem_id'] == row['problem_id']) & (df['session_id'] != row['session_id'])]
        if len(same_problem) > 0:
            return same_problem[value_col].mean()
        else:
            return df[df['session_id'] != row['session_id']][value_col].mean()
    
    result_df['problem_mean'] = df.apply(calc_problem_loo, axis=1)
    
    # ユーザー平均（LOO）
    def calc_user_loo(row):
        other_problems = df[(df['session_id'] == row['session_id']) & (df['problem_id'] != row['problem_id'])]
        if len(other_problems) > 0:
            return other_problems[value_col].mean()
        else:
            return df[df['session_id'] != row['session_id']][value_col].mean()
    
    result_df['user_mean'] = df.apply(calc_user_loo, axis=1)
    
    return result_df

print("ベースライン計算中...")

## 5. 自信度予測の検証（考察.md 2.2節）

期待値:
- LLM: MAE=1.09, r=0.600, R²=0.360, 完全一致=32.7%, ±1以内=63.7%
- 全体平均: MAE=1.65, r=0, 完全一致=8%, ±1以内=22%
- 問題平均: MAE=1.35, r=0.467, 完全一致=16%, ±1以内=48%
- ユーザー平均: MAE=1.30, r=0.457, 完全一致=21.3%, ±1以内=44.7%

In [ ]:
print("=" * 50)
print("📊 自信度予測の検証")
print("=" * 50)

# 自信度のベースライン計算
conf_df = calc_loo_baselines(df, 'confidence')

# LLM予測
llm_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['ai_predicted_confidence'])
print_metrics("LLM予測", llm_conf_metrics)

# 全体平均
global_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['global_mean'])
print_metrics("全体平均", global_conf_metrics)

# 問題平均
problem_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['problem_mean'])
print_metrics("問題平均", problem_conf_metrics)

# ユーザー平均
user_conf_metrics = calculate_metrics(conf_df['confidence'], conf_df['user_mean'])
print_metrics("ユーザー平均", user_conf_metrics)

## 6. 挑戦度予測の検証（考察.md 2.3節）

期待値:
- LLM: MAE=1.23, r=0.371
- 全体平均: MAE=1.18
- 問題平均: MAE=0.94, r=0.500
- ユーザー平均: MAE=1.01, r=0.347

In [ ]:
print("=" * 50)
print("📊 挑戦度予測の検証")
print("=" * 50)

# 挑戦度のベースライン計算
chal_df = calc_loo_baselines(df, 'challenge')

# LLM予測
llm_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['ai_predicted_challenge'])
print_metrics("LLM予測", llm_chal_metrics)

# 全体平均
global_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['global_mean'])
print_metrics("全体平均", global_chal_metrics)

# 問題平均
problem_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['problem_mean'])
print_metrics("問題平均", problem_chal_metrics)

# ユーザー平均
user_chal_metrics = calculate_metrics(chal_df['challenge'], chal_df['user_mean'])
print_metrics("ユーザー平均", user_chal_metrics)

## 7. GAP予測の検証（考察.md 2.4節）

期待値:
- LLM: MAE=2.11, r=0.532, R²=0.283, 完全一致=21.7%, ±1以内=42.3%
- 全体平均: MAE=2.68, 完全一致=4%, ±1以内=7%
- 問題平均: MAE=2.07, r=0.513
- ユーザー平均: MAE=2.20, r=0.403

In [ ]:
print("=" * 50)
print("📊 GAP予測の検証")
print("=" * 50)

# GAPのベースライン計算
gap_df = calc_loo_baselines(df, 'human_gap')

# LLM予測
llm_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['ai_gap'])
print_metrics("LLM予測", llm_gap_metrics)

# 全体平均
global_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['global_mean'])
print_metrics("全体平均", global_gap_metrics)

# 問題平均
problem_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['problem_mean'])
print_metrics("問題平均", problem_gap_metrics)

# ユーザー平均
user_gap_metrics = calculate_metrics(gap_df['human_gap'], gap_df['user_mean'])
print_metrics("ユーザー平均", user_gap_metrics)

## 8. 混合効果モデル分析（考察.md 3.2節）

期待値:
- 問題効果 (ICC): 27.1%
- ユーザー効果 (ICC): 10.2%
- 残差: 62.7%
- LLM係数 (β): 0.524, p < 0.001
- 周辺R²: 10.0%
- 条件付きR²: 37.3%

In [ ]:
print("=" * 50)
print("📊 混合効果モデル分析")
print("=" * 50)

# データ準備
model_df = df[['human_gap', 'ai_gap', 'problem_id', 'session_id']].copy()
model_df = model_df.rename(columns={'session_id': 'user_id'})
model_df = model_df.dropna()

# Full Model
full_model = smf.mixedlm(
    "human_gap ~ ai_gap", 
    data=model_df, 
    groups=model_df["user_id"],
    re_formula="~1",
    vc_formula={"problem_id": "0 + C(problem_id)"}
)
full_result = full_model.fit()

# 分散成分の抽出
sigma2_user = full_result.cov_re.iloc[0, 0]
sigma2_problem = full_result.vcomp[0] if len(full_result.vcomp) > 0 else 0
sigma2_residual = full_result.scale
total_variance = sigma2_problem + sigma2_user + sigma2_residual

# ICC
icc_problem = sigma2_problem / total_variance
icc_user = sigma2_user / total_variance
icc_residual = sigma2_residual / total_variance

print(f"\n分散成分:")
print(f"  問題効果 (ICC): {icc_problem:.1%}")
print(f"  ユーザー効果 (ICC): {icc_user:.1%}")
print(f"  残差: {icc_residual:.1%}")

# 固定効果
beta_ai = full_result.params['ai_gap']
p_ai = full_result.pvalues['ai_gap']

print(f"\n固定効果:")
print(f"  LLM係数 (β): {beta_ai:.3f}")
print(f"  p値: {p_ai:.4g}")

# R² (Nakagawa & Schielzeth, 2013)
fixed_pred = full_result.params['Intercept'] + full_result.params['ai_gap'] * model_df['ai_gap']
var_fixed = fixed_pred.var()
var_random = sigma2_problem + sigma2_user
denom = var_fixed + var_random + sigma2_residual

marginal_r2 = var_fixed / denom
conditional_r2 = (var_fixed + var_random) / denom

print(f"\nR²:")
print(f"  周辺R² (固定効果のみ): {marginal_r2:.1%}")
print(f"  条件付きR² (全効果): {conditional_r2:.1%}")

## 9. 残差診断（考察.md 3.3節）

期待値:
- 平均: -0.0000
- 歪度: -0.33
- 尖度: -0.21
- W統計量: 0.9875, p = 0.011

In [ ]:
print("=" * 50)
print("📊 残差診断")
print("=" * 50)

# 残差の計算
residuals = model_df['human_gap'] - full_result.fittedvalues

print(f"\n残差の統計量:")
print(f"  平均: {residuals.mean():.4f}")
print(f"  標準偏差: {residuals.std():.4f}")
print(f"  歪度: {stats.skew(residuals):.4f}")
print(f"  尖度: {stats.kurtosis(residuals):.4f}")

# Shapiro-Wilk検定
shapiro_stat, shapiro_p = shapiro(residuals)
print(f"\nShapiro-Wilk検定:")
print(f"  W = {shapiro_stat:.4f}")
print(f"  p = {shapiro_p:.4f}")

## 10. ピアソン vs スピアマン相関の比較（考察.md 3.4節）

期待値:
- 自信度: Pearson=0.600, Spearman=0.576, 差=0.024
- 挑戦度: Pearson=0.371, Spearman=0.360, 差=0.011
- GAP: Pearson=0.532, Spearman=0.507, 差=0.025

In [ ]:
print("=" * 50)
print("📊 ピアソン vs スピアマン相関の比較")
print("=" * 50)

comparisons = [
    ("自信度", "confidence", "ai_predicted_confidence"),
    ("挑戦度", "challenge", "ai_predicted_challenge"),
    ("GAP", "human_gap", "ai_gap"),
]

print(f"\n{'指標':<10} {'ピアソン(r)':<12} {'スピアマン(ρ)':<14} {'差':<8} {'判定'}")
print("-" * 55)

for name, human_col, ai_col in comparisons:
    pearson_r, _ = pearsonr(df[human_col], df[ai_col])
    spearman_r, _ = spearmanr(df[human_col], df[ai_col])
    diff = abs(pearson_r - spearman_r)
    judge = "✅ 一致" if diff < 0.05 else "⚠️ 乖離"
    print(f"{name:<10} {pearson_r:<12.3f} {spearman_r:<14.3f} {diff:<8.3f} {judge}")

## 11. 人間の自信度と挑戦度の相関

In [ ]:
print("=" * 50)
print("📊 人間の自信度と挑戦度の相関")
print("=" * 50)

conf_chal_r, conf_chal_p = pearsonr(df['confidence'], df['challenge'])
print(f"\nPearson相関係数: r = {conf_chal_r:.4f}")
print(f"p値: {conf_chal_p:.4g}")
print(f"決定係数 (R²): {conf_chal_r**2:.4f}")

if abs(conf_chal_r) >= 0.7:
    strength = "強い"
elif abs(conf_chal_r) >= 0.4:
    strength = "中程度の"
else:
    strength = "弱い"
direction = "正の" if conf_chal_r > 0 else "負の"

print(f"\n解釈: {strength}{direction}相関")

## 12. 結果サマリー

In [ ]:
print("="*60)
print("📋 考察.mdとの比較サマリー")
print("="*60)

print("\n【基本統計】")
print(f"  セッション数: {len(sessions_df)} (期待値: 11)")
print(f"  回答数: {len(df)} (期待値: 300)")
print(f"  問題数: {df['problem_id'].nunique()} (期待値: 35)")
print(f"  ユーザー数: {df['session_id'].nunique()} (期待値: 10)")

print("\n【自信度予測 - LLM】")
print(f"  MAE: {llm_conf_metrics['mae']:.2f} (期待値: 1.09)")
print(f"  r: {llm_conf_metrics['r']:.3f} (期待値: 0.600)")
print(f"  完全一致率: {llm_conf_metrics['exact_match']:.1f}% (期待値: 32.7%)")

print("\n【挑戦度予測 - LLM】")
print(f"  MAE: {llm_chal_metrics['mae']:.2f} (期待値: 1.23)")
print(f"  r: {llm_chal_metrics['r']:.3f} (期待値: 0.371)")

print("\n【GAP予測 - LLM】")
print(f"  MAE: {llm_gap_metrics['mae']:.2f} (期待値: 2.11)")
print(f"  r: {llm_gap_metrics['r']:.3f} (期待値: 0.532)")

print("\n【混合効果モデル】")
print(f"  問題効果 (ICC): {icc_problem:.1%} (期待値: 27.1%)")
print(f"  ユーザー効果 (ICC): {icc_user:.1%} (期待値: 10.2%)")
print(f"  LLM係数 (β): {beta_ai:.3f} (期待値: 0.524)")
print(f"  周辺R²: {marginal_r2:.1%} (期待値: 10.0%)")